### Structured Output
##### Models can be requested to provide their response in a format matching given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

### Pydantic 
#### Pydantic models provide the richest feature set with field validation,descriptions,and nested structures.


In [7]:
# Step 1: load the environment variables from ".env" file 
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model=init_chat_model("groq:qwen/qwen3-32b")  # Here, this model without any structure
model


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001B3EB5F6E40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B3EBD7BD70>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# Step 2: Import Basemodels fro Pydantic 

from pydantic import BaseModel,Field  # this field is for field validation
# LLM to follow some structured schema. For this create a class and inherit the basemodel
# This is the output my LLM going to generate (title,year,director,rating)

class Movie(BaseModel):
    """ A movie with details"""
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

model_with_strucure=model.with_structured_output(Movie)  #here, we attached a strucure to model
model_with_strucure


RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001B3EB5F6E40>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B3EBD7BD70>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', '

In [ ]:
# step 3: Invoking the model,we get the default output but we want strucured output. Call with model what we created in next cell
model.invoke("Provide me Inception movie details")

AIMessage(content='<think>\nOkay, the user wants details about the movie Inception. Let me start by recalling the basic information. The movie was directed by Christopher Nolan, right? The release year was 2010. Main actors include Leonardo DiCaprio as Dom Cobb, and then there\'s Joseph Gordon-Levitt, Ellen Page, Tom Hardy, and others. The genre is sci-fi, action, thriller.\n\nI should mention the plot summary. The main idea is about entering dreams to plant ideas. The concept of heists in the dream world. The use of the machine called the PASIV device. The main character, Dom Cobb, is a thief who steals secrets by infiltrating the subconscious. The team\'s plan to perform an inception, which is planting an idea instead of stealing it.\n\nKey themes: reality vs. dreams, time perception, guilt, family. The rotating hallway fight scene with Joseph Gordon-Levitt is iconic. The ending with the spinning top is ambiguous, leaving the audience wondering if it\'s a dream or reality.\n\nTechnic

In [ ]:
# to get the strucured output, we have to use our model with structure
model_with_strucure.invoke("Provide me Inception movie details")

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

In [11]:
model_with_strucure.invoke("Provide me about RRR movie details")

Movie(title='RRR', year=2022, director='S.S. Rajamouli', rating=8.8)

In [ ]:
# sometimes we also need the raw reponse along with structured output 

### (Raw) Message output alongside parsed structure

In [23]:
class Movie_1(BaseModel):
    """ A movie with details"""
    title:str=Field(...,description="The title of the movie")  # ... for optional fields
    year:int=Field(...,description="This year the movie was released")
    director:str=Field(...,description="The director of the movie")
    rating:float=Field(...,description="The movies rating out of 10") 

model_with_strucure=model.with_structured_output(Movie,include_raw=True)  #here, we attached a strucure to model
model_with_strucure.invoke("Provide details about movie Inception")

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Inception. Let me check the available tools. There\'s a Movie function that requires title, year, director, and rating. I need to recall the information for Inception. The title is "Inception", directed by Christopher Nolan, released in 2010, and it has a high rating. Let me confirm the exact year. Oh right, it\'s 2010. The rating is around 8.8 on IMDb. I should structure the arguments with these details. Make sure all required parameters are included. Yep, title, year, director, and rating. Let me format the JSON correctly.\n', 'tool_calls': [{'id': '94pw2qekd', 'function': {'arguments': '{"director":"Christopher Nolan","rating":8.8,"title":"Inception","year":2010}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 186, 'prompt_tokens': 224, 'total_tokens': 410, 'completion_time': 0.311377102, 'completion_tokens_det

### Nested Structure

In [24]:
from pydantic import BaseModel,Field

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str 
    year: int
    cast: list[Actor]   #using the Actor class
    genre: list[str]
    budget: float | None=Field(None,description="Budget in million USD")

In [26]:
model_with_strucure=model.with_structured_output(MovieDetails)
model_with_strucure.invoke("Provide details about movie Inception")

MovieDetails(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Elliot Page', role='Ariadne'), Actor(name='Tom Hardy', role='Balthazar')], genre=['Science Fiction', 'Action'], budget=160.0)

In [27]:
model_with_strucure.invoke("Provide details about movie Interstellar")

MovieDetails(title='Interstellar', year=2014, cast=[Actor(name='Matthew McConaughey', role='Cooper'), Actor(name='Anne Hathaway', role='Amelia Brand'), Actor(name='Jessica Chastain', role='Murph'), Actor(name='Michael Caine', role='Professor Brand')], genre=['Science Fiction', 'Adventure'], budget=165.0)

### TypedDict
##### TypedDict provides a simpler alternative using Python's built-in typing, ideal when you don't need runtime validation

In [31]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """ A movie with details"""
    title:Annotated[str,...,"The title of the movie"]           # ... for optional fields we can keep them as empty
    year:Annotated[int,...,"This year the movie was released"]
    director:Annotated[str,...,"The director of the movie"]
    rating:Annotated[float,...,"The movies rating out of 10"]

model_with_typeddict=model.with_structured_output(MovieDict)       #here, we attached a typedict to model
model_with_typeddict.invoke("Provide details about movie Avengers")

{'director': 'Joss Whedon', 'rating': 8, 'title': 'Avengers', 'year': 2012}

In [33]:
model_with_typeddict.invoke("Provide details about movie Bahubali")

{'director': 'S.S. Rajamouli',
 'rating': 8.3,
 'title': 'Bahubali',
 'year': 2015}

In [36]:
# same using nested structure

class Actor(TypedDict):
    name: str
    role: str

class MovieDetails(TypedDict):
    title: str 
    year: int
    cast: list[Actor]   #using the Actor class
    genre: list[str]
    budget: float | None=Field(None,description="Budget in million USD")

model_with_typeddict_1=model.with_structured_output(MovieDetails)
model_with_typeddict_1.invoke("Provide details about movie Inception")

{'budget': 160000000,
 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Dom Cobb'},
  {'name': 'Joseph Gordon-Levitt', 'role': 'Arthur'},
  {'name': 'Elliot Page', 'role': 'Ariadne'},
  {'name': 'Tom Hardy', 'role': 'Eames'}],
 'genre': ['Action', 'Sci-Fi', 'Thriller'],
 'title': 'Inception',
 'year': 2010}

In [38]:
model_with_typeddict_1.profile

AttributeError: 'RunnableSequence' object has no attribute 'profile'

In [39]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

### DataClasses 
##### A data class is a class typically containing mainly data, although there aren't really any restrictions. You create it using the @dataclass decorator.


### Creating an agent using Pydantic

In [44]:
# Call OPENAI_API_KEY first to use gpt-5 model
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

from pydantic import BaseModel,Field
from langchain.agents import create_agent

# Creating a schema 
class ContactInfo(BaseModel):   # Whenever we use BaseModel, it is for validation using Pydantic
    """Contact information for a person"""
    name:str=Field(description="The name of the person")
    email:str=Field(description="The email of the person")
    phone:str=Field(description="The phone number of the person")

# Creating an agent
agent=create_agent(
    model="gpt-5",
    response_format=ContactInfo   # this agent's response is in ContactInfo schema
    )

# Calling an agent 
result = agent.invoke({
    "messages":[{"role":"user","content":"Extract contact info from: Venkat, venkat@example.com, (901) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='Venkat', email='venkat@example.com', phone='(901) 123-4567')

### Creating an agent using TypedDict

In [ ]:
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

from typing_extensions import TypedDict
from langchain.agents import create_agent

# Creating a schema 
class ContactInfo(TypedDict):   # Whenever we use TypedDict, not for runtime validation (unlike Pydantic)
    """Contact information for a person"""
    name:str=Field(description="The name of the person")
    email:str=Field(description="The email of the person")
    phone:str=Field(description="The phone number of the person")

# Creating an agent
agent=create_agent(
    model="gpt-5",
    response_format=ContactInfo   # this agent's response is in ContactInfo schema
    )

# Calling an agent 
result = agent.invoke({
    "messages":[{"role":"user","content":"Extract contact info from: Venkat, venkat@example.com, (901) 123-4567"}]
})

result["structured_response"]
#output in Dict format : {'name': 'Venkat', 'email': 'venkat@example.com', 'phone': '(901) 123-4567'}

{'name': 'Venkat', 'email': 'venkat@example.com', 'phone': '(901) 123-4567'}

### With Dataclass

In [46]:
from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    # creating variables 
    name:str  # for the name of the person
    email:str # for an email of the person
    phone:str # for the phone number of the person

# create an agent 
agent=create_agent(
    model="gpt-5",
    response_format=ContactInfo
)

# Calling an agent 
result = agent.invoke({
    "messages":[{"role":"user","content":"Extract contact info from: Venkat, venkat@example.com, (901) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='Venkat', email='venkat@example.com', phone='(901) 123-4567')